# 🚀 ML Trading Japan - 快速入门教程

欢迎使用 ML Trading Japan！这个笔记本将带您在 **10 分钟内** 了解核心功能。

## 📚 本教程内容

1. 🔧 环境设置和导入
2. 📊 生成示例数据
3. ⚙️ 创建 63 个技术指标
4. 🎯 配置市场参数
5. 🤖 准备机器学习特征

## 🎯 学习目标

通过本教程，您将学会：
- 如何使用 FeatureEngineer 创建技术指标
- 如何配置不同市场的参数
- 如何准备数据用于机器学习模型

让我们开始吧！🎉

## 1. 环境设置和导入 🔧

首先，让我们导入所需的库并设置路径。

In [ ]:
import sys
from pathlib import Path

# 添加父目录到路径
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 导入我们的框架
from ml_framework import (
    FeatureEngineer,
    MLSignalPredictor,
    Backtester,
    get_market_config
)

# 设置绘图样式
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ 所有库导入成功！")
print(f"   Pandas 版本: {pd.__version__}")
print(f"   NumPy 版本: {np.__version__}")

## 2. 生成示例数据 📊

让我们创建一些模拟的 1 分钟 K 线数据。在实际应用中，您会从交易所或数据供应商获取真实数据。

In [ ]:
# 设置随机种子以保证可重复性
np.random.seed(42)

# 创建日期范围（2天的1分钟数据）
dates = pd.date_range('2024-01-01 09:00', periods=1000, freq='1min')

# 生成OHLCV数据
base_price = 2400
price_changes = np.random.randn(1000) * 2
closes = base_price + np.cumsum(price_changes)

df = pd.DataFrame({
    'datetime': dates,
    'open': closes + np.random.randn(1000) * 0.5,
    'high': closes + np.abs(np.random.randn(1000) * 3),
    'low': closes - np.abs(np.random.randn(1000) * 3),
    'close': closes,
    'volume': np.random.randint(1000, 50000, 1000)
})

print(f"✅ 生成了 {len(df):,} 行数据")
print(f"   时间范围: {df['datetime'].min()} 到 {df['datetime'].max()}")
print(f"   价格范围: ¥{df['close'].min():.0f} - ¥{df['close'].max():.0f}")
print(f"\n前5行数据:")
df.head()

In [ ]:
# 可视化价格数据
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# 价格图
axes[0].plot(df['datetime'], df['close'], label='收盘价', linewidth=1.5)
axes[0].fill_between(df['datetime'], df['low'], df['high'], alpha=0.2, label='高低价区间')
axes[0].set_title('TOPIX 价格走势', fontsize=14, fontweight='bold')
axes[0].set_ylabel('价格 (¥)', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 成交量图
axes[1].bar(df['datetime'], df['volume'], width=0.0007, alpha=0.6, color='steelblue')
axes[1].set_title('成交量', fontsize=14, fontweight='bold')
axes[1].set_xlabel('时间', fontsize=12)
axes[1].set_ylabel('成交量', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 数据可视化完成！")

## 3. 创建技术指标 ⚙️

现在让我们使用 `FeatureEngineer` 创建 **63 个技术指标**！

这些指标包括：
- 📈 **趋势指标**: EMA, MACD, ADX
- 🎯 **动量指标**: RSI, Stochastic, CCI, Williams %R
- 📊 **波动率指标**: ATR, Bollinger Bands
- 💹 **成交量指标**: OBV, Volume SMA
- 🔍 **市场微观结构**: 价格比率、影线分析

In [ ]:
# 创建特征工程器
engineer = FeatureEngineer()

# 生成所有特征
print("⏳ 正在创建 63 个技术指标...")
df_with_features = engineer.create_all_features(df)

# 清理 NaN 值
initial_len = len(df_with_features)
df_clean = df_with_features.dropna()

print(f"\n✅ 特征创建完成！")
print(f"   原始行数: {initial_len:,}")
print(f"   清理后行数: {len(df_clean):,}")
print(f"   移除 NaN: {initial_len - len(df_clean):,} 行")
print(f"   总特征数: {len(df_clean.columns):,} 列")

print(f"\n📊 部分特征列表:")
feature_cols = [c for c in df_clean.columns if c not in ['datetime', 'open', 'high', 'low', 'close', 'volume']]
for i, col in enumerate(feature_cols[:15], 1):
    print(f"   {i:2d}. {col}")
print(f"   ... 还有 {len(feature_cols) - 15} 个特征")

In [ ]:
# 可视化几个重要指标
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# RSI
axes[0].plot(df_clean['datetime'], df_clean['rsi_14'], label='RSI(14)', color='purple', linewidth=1.5)
axes[0].axhline(y=70, color='r', linestyle='--', alpha=0.5, label='超买(70)')
axes[0].axhline(y=30, color='g', linestyle='--', alpha=0.5, label='超卖(30)')
axes[0].set_title('RSI 相对强弱指标', fontsize=12, fontweight='bold')
axes[0].set_ylabel('RSI', fontsize=10)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MACD
axes[1].plot(df_clean['datetime'], df_clean['macd'], label='MACD', color='blue', linewidth=1.5)
axes[1].plot(df_clean['datetime'], df_clean['macd_signal'], label='Signal', color='red', linewidth=1.5)
axes[1].bar(df_clean['datetime'], df_clean['macd_hist'], alpha=0.3, label='Histogram')
axes[1].set_title('MACD 指标', fontsize=12, fontweight='bold')
axes[1].set_ylabel('MACD', fontsize=10)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Bollinger Bands
axes[2].plot(df_clean['datetime'], df_clean['close'], label='收盘价', color='black', linewidth=1.5)
axes[2].plot(df_clean['datetime'], df_clean['bb_upper_20'], label='上轨', color='red', linestyle='--', alpha=0.7)
axes[2].plot(df_clean['datetime'], df_clean['bb_middle_20'], label='中轨', color='gray', linestyle='--', alpha=0.7)
axes[2].plot(df_clean['datetime'], df_clean['bb_lower_20'], label='下轨', color='green', linestyle='--', alpha=0.7)
axes[2].fill_between(df_clean['datetime'], df_clean['bb_lower_20'], df_clean['bb_upper_20'], alpha=0.1)
axes[2].set_title('布林带 (Bollinger Bands)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('时间', fontsize=10)
axes[2].set_ylabel('价格', fontsize=10)
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ 技术指标可视化完成！")

## 4. 市场配置 🎯

让我们加载市场配置。框架支持多个市场：
- **TOPIX Future** (tick=0.5, multiplier=10,000)
- **Nikkei225 Mini** (tick=10, multiplier=1,000)

In [ ]:
# 加载 TOPIX 配置
config = get_market_config('topix')

print("✅ 市场配置加载完成！")
print(f"\n市场名称: {config.name}")
print(f"最小变动单位: {config.tick_size}")
print(f"合约乘数: {config.multiplier:,}")
print(f"初始资金: ¥{config.initial_capital:,}")

# 计算盈亏示例
print(f"\n💰 盈亏计算示例:")
entry_price = 2400.0
exit_price = 2410.0
pnl = config.calculate_pnl(entry_price, exit_price, position=1)
print(f"   买入价: ¥{entry_price:.1f}")
print(f"   卖出价: ¥{exit_price:.1f}")
print(f"   盈亏: ¥{pnl:,.0f}")

## 5. 准备 ML 特征 🤖

最后一步：准备数据用于机器学习模型。

In [ ]:
# 选择特征列（排除OHLCV和时间）
exclude_cols = ['datetime', 'open', 'high', 'low', 'close', 'volume']
feature_cols = [c for c in df_clean.columns if c not in exclude_cols]

# 准备特征矩阵
X = df_clean[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)

print("✅ ML 特征准备完成！")
print(f"\n📊 特征矩阵信息:")
print(f"   形状: {X.shape}")
print(f"   样本数: {X.shape[0]:,}")
print(f"   特征数: {X.shape[1]}")
print(f"   缺失值: {X.isna().sum().sum()}")
print(f"   无穷值: {np.isinf(X.values).sum()}")

print(f"\n🎉 数据已准备就绪，可以用于训练机器学习模型！")

# 显示特征统计
print(f"\n📈 特征统计摘要:")
X.describe().T.head(10)

## 🎊 恭喜！教程完成

您已经成功完成快速入门教程！

### 📚 下一步

现在您已经了解了基础知识，可以继续：

1. **02_feature_engineering.ipynb** - 深入了解 63 个技术指标
2. **03_model_training.ipynb** - 训练 XGBoost 模型
3. **04_optimization.ipynb** - Walk-Forward 参数优化
4. **05_performance_analysis.ipynb** - 性能分析和结果可视化

### 💡 关键要点

✅ **FeatureEngineer** 可以自动创建 63 个技术指标  
✅ 框架支持多个市场配置（TOPIX, Nikkei225）  
✅ 数据清理和特征准备流程简单  
✅ 所有特征都经过 NaN/Inf 处理  

### 🚀 实战应用

查看完整示例：
- `examples/topix_example.py` - TOPIX 完整交易流程
- `examples/run_optimization.py` - 参数优化实战

---

**Happy Trading!** 🎉📈